# Chat Based Recommendation System

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 25.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 88.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.3 MB/s eta 0:00:00


In [19]:
import pandas as pd
from transformers import GPT2LMHeadModel, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup
import torch

def load_and_preprocess_data(file_path):
    df = pd.read_csv(file_path)

    selected_columns = ['Product', 'Main Category', 'Sub Category', 'Actual Price' ]

    text_data = df[selected_columns].apply(lambda row: ' , '.join(map(str, row)), axis=1)

    text_data = "\n".join(text_data)

    return text_data


data_path = "/content/sports.csv"
text_data = load_and_preprocess_data(data_path)

model_name = "gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)


max_seq_length = 256
input_ids = tokenizer.encode(text_data, return_tensors='pt', max_length=max_seq_length, truncation=True)


learning_rate = 2e-4
epochs = 15
batch_size = 8

optimizer = AdamW(model.parameters(), lr=learning_rate)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=len(input_ids) * epochs)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(epochs):
    model.train()
    for i in range(0, len(input_ids), batch_size):
        batch_input_ids = input_ids[i:i+batch_size].to(device)
        optimizer.zero_grad()

        outputs = model(input_ids=batch_input_ids, labels=batch_input_ids)
        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()

        if i % 100 == 0:
            print(f"Epoch [{epoch+1}/{epochs}] - Batch [{i}/{len(input_ids)}] - Loss: {loss.item()}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch [1/15] - Batch [0/1] - Loss: 4.297889232635498
Epoch [2/15] - Batch [0/1] - Loss: 3.2711050510406494
Epoch [3/15] - Batch [0/1] - Loss: 2.777663230895996
Epoch [4/15] - Batch [0/1] - Loss: 2.349323034286499
Epoch [5/15] - Batch [0/1] - Loss: 1.9780586957931519
Epoch [6/15] - Batch [0/1] - Loss: 1.654369831085205
Epoch [7/15] - Batch [0/1] - Loss: 1.3463140726089478
Epoch [8/15] - Batch [0/1] - Loss: 1.194441795349121
Epoch [9/15] - Batch [0/1] - Loss: 0.8933115601539612
Epoch [10/15] - Batch [0/1] - Loss: 0.7515425086021423
Epoch [11/15] - Batch [0/1] - Loss: 0.6275886297225952
Epoch [12/15] - Batch [0/1] - Loss: 0.5562211871147156
Epoch [13/15] - Batch [0/1] - Loss: 0.45950597524642944
Epoch [14/15] - Batch [0/1] - Loss: 0.4424612522125244
Epoch [15/15] - Batch [0/1] - Loss: 0.38575735688209534


In [20]:
model.save_pretrained("gpt2-trained-sports")

print("Trained model saved at the default location")


Trained model saved at the default location


In [24]:
import torch
from transformers import GPT2LMHeadModel, GPT2Tokenizer

model_path = "gpt2-trained-sports"

tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained(model_path)

def generate_recommendation(user_query, max_response_length=50):
    input_ids = tokenizer.encode(user_query, return_tensors="pt")

    with torch.no_grad():
        output_ids = model.generate(input_ids, max_length=max_response_length, num_return_sequences=1)

    generated_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return generated_text

input_text = "best cricket kit for boys?"
restaurant_recommendation = generate_recommendation(input_text)
print(restaurant_recommendation)


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


best cricket kit for boys?

I think it's a good idea.

I think it's a good idea.

I think it's a good idea.

I think it's a good idea.

I think
